# CS Prediction in VR

## Author: 
###Tiago Manuel Severino Gonçalves. IST Lisboa, Portugal. Computer Engineering student number 89547

## Library imports

Import all the necessary libraries

* [numpy](https://numpy.org/): we will use it to store the data in array format for visualization
* [pandas](https://pandas.pydata.org/): fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language. 
* [sklearn](https://scikit-learn.org/): provides a [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) implementation that we will use for visualization
* [matplotlib](https://matplotlib.org/): plotting library for visualization
* [tensorflow](https://www.tensorflow.org/): the neural network library

In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, LeaveOneOut, ShuffleSplit, StratifiedShuffleSplit, RepeatedStratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier # Will later be deprecated and should be replaced with scikeras (https://github.com/adriangb/scikeras)

import multiprocessing
print(multiprocessing.cpu_count())

rootFolder = "/content/drive/MyDrive/Tese/" #change this to better suit your needs

print("tensorflow version",tf.__version__)
print("sklearn version",sklearn.__version__)
print("keras version",tf.keras.__version__)


2
tensorflow version 2.9.2
sklearn version 1.0.2
keras version 2.9.0


## Data Preperation

#### In subsequent runs of the program, this step can be skipped during later testing, provided you have run it at least once as it saves both the labels and the features in a suitable format, such as csv or a readable numpy array

### Labels

##### Now let's get all the valid experiment identifiers from FMS_tag.csv as well as the labels for each timestep of features

In [ ]:
with open(rootFolder+"/CSV_Data/TAGS/FMS_Tag.csv","r") as fmsTags:
  fmsTagsDict = {}

  lines = fmsTags.readlines()

  for line in lines:
    labels = []
    tokens = line.split(',')
    for token in tokens[1:]:
      labels.append(int(token))
    fmsTagsDict[tokens[0]] = labels
  
keys = fmsTagsDict.keys()

""" The following converts the dictionary into a digestible csv file, with every
    empty entry being filled with an Nan"""
fmsTagsDF = pd.DataFrame({key: pd.Series(value) for key, value in fmsTagsDict.items()})
fmsTagsDF.to_csv(rootFolder+"/CSV_Data/LABELS/labels.csv", encoding='utf-8', index=False)

""" The following is how to get a list of the labels for a certain key (in this case, '0')
    without all the NaN values. It also converts all the values to int which is what we want"""
#temp = fmsTagsDF['0'].dropna().astype(int).tolist()

"""
# locate in the labelsDF a label for a given key (ex '0') and minute (ex 18)
# print(int(labelsDF['0'].iloc[18]))

# or use a dict from the labelsDF

labelsDict = {}
for columnName, columnData in labelsDF.iteritems():
  labelsDict[columnName] = labelsDF[columnName].dropna().astype(int).tolist()

"""

"\n# locate in the labelsDF a label for a given key (ex '0') and minute (ex 18)\n# print(int(labelsDF['0'].iloc[18]))\n\n# or use a dict from the labelsDF\n\nlabelsDict = {}\nfor columnName, columnData in labelsDF.iteritems():\n  labelsDict[columnName] = labelsDF[columnName].dropna().astype(int).tolist()\n\n"

### Video Processing

##### Here we process the video to remove features for the different approaches of Machine Learning, tradional and deep learning.

In [ ]:
import cv2 #OpenCv package
import os
from scipy.stats import skew
from scipy.stats import kurtosis
from skimage.measure import shannon_entropy

videosDirectory = rootFolder+"/RECORDINGS/"

# Deep Learning features

deepLearningList = []
totalMinutes = 0

for key in keys: # keys being the experiments that were considered valid
    filename = key +'.mp4' 
    title = filename.split('.')[0]
    filename = videosDirectory + '/' + filename
    video = cv2.VideoCapture(filename)

    frameCount = 0
    frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = int(video.get(cv2.CAP_PROP_FPS))
    duration = int(frames/fps)
    timestep = 60
    framesPerTimestep = fps * timestep
        
    print(filename,duration//60,frames)

    width = int(video.get(3))
    height = int(video.get(4))

    # HSV initialization
    hueList = []
    satList = []
    brightList = []

    hue = {}
    [hue.setdefault(i, 0) for i in range(32)] 
    sat = {}
    [sat.setdefault(i, 0) for i in range(32)] 
    bright = {}
    [bright.setdefault(i, 0) for i in range(32)] 

    # Motion Intensity Initialization

    flowList = []
    meanFlowList = []
    stdDevFlowList = []
    skewnessFlowList = []
    kurtosisFlowList = []
    maxFlowList = []
    minFlowList = []
    varianceFlowList = []

    # Contrast Initialization

    contrastList = []
    meanContrastList = []
    stdDevContrastList = []
    skewnessContrastList = []
    kurtosisContrastList = []
    maxContrastList = []
    minContrastList = []
    varianceContrastList = []

    # Entropy Initialization

    entropyList = []
    meanEntropyList = []
    stdDevEntropyList = []
    skewnessEntropyList = []
    kurtosisEntropyList = []
    maxEntropyList = []
    minEntropyList = []
    varianceEntropyList = []

    while True:
        elapsedSeconds = frameCount/fps
        #print("frameCount:",frameCount, "  elapsedSeconds: ", elapsedSeconds)

        ret, frameImg = video.read()
        frameCount+=1

        #if(elapsedSeconds > 120): break

        if(elapsedSeconds%timestep == 0 and elapsedSeconds > 0):
            print("     TIMESTEP: ", elapsedSeconds/timestep)

            # HSV statistics
            hueList.append(hue)
            satList.append(sat)
            brightList.append(bright)

            # Motion Intensity statistics
            meanFlowList.append(np.mean(flowList))
            stdDevFlowList.append(np.std(flowList))
            skewnessFlowList.append(skew(flowList))
            kurtosisFlowList.append(kurtosis(flowList))
            maxFlowList.append(max(flowList))
            minFlowList.append(min(flowList))
            varianceFlowList.append(np.var(flowList))

            # Contrast statistics
            meanContrastList.append(np.mean(contrastList))
            stdDevContrastList.append(np.std(contrastList))
            skewnessContrastList.append(skew(contrastList))
            kurtosisContrastList.append(kurtosis(contrastList))
            maxContrastList.append(max(contrastList))
            minContrastList.append(min(contrastList))
            varianceContrastList.append(np.var(contrastList))


            # Entropy statistics
            meanEntropyList.append(np.mean(entropyList))
            stdDevEntropyList.append(np.std(entropyList))
            skewnessEntropyList.append(skew(entropyList))
            kurtosisEntropyList.append(kurtosis(entropyList))
            maxEntropyList.append(max(entropyList))
            minEntropyList.append(min(entropyList))
            varianceEntropyList.append(np.var(entropyList))

            # Deep Learning features

            deepLearningDFTemp = pd.DataFrame([flowList,contrastList,entropyList])
            deepLearningDFTemp = deepLearningDFTemp.transpose()
            deepLearningDFTemp.columns=['Flow','Contrast','Entropy']

            deepLearningList.append(deepLearningDFTemp.to_numpy())

            deepLearningDFTemp = pd.DataFrame()

            # Reset temporary variables

            hue = {}
            [hue.setdefault(i, 0) for i in range(32)] 
            sat = {}
            [sat.setdefault(i, 0) for i in range(32)] 
            bright = {}
            [bright.setdefault(i, 0) for i in range(32)]

            flowList = []
            contrastList = []
            entropyList = []

        if(elapsedSeconds >= duration): break

        # Resize so that its less computationally intensive to process

        frameImg = cv2.resize(frameImg, (128, 72))

        # HSV Capture

        hsv = cv2.cvtColor(frameImg, cv2.COLOR_BGR2HSV)
        h,s,v = cv2.split(hsv)
        for j in range(len(h)):
            for k in range(len(h[j])):
                hue[h[j][k]//8] += 1
                sat[s[j][k]//8] += 1
                bright[v[j][k]//8] += 1

        # Motion Intensity Capture

        if(elapsedSeconds > 0):
            gray = cv2.cvtColor(frameImg, cv2.COLOR_BGR2GRAY)
            flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            prevgray = gray

            # We use the mean of the flow because why not and I don't think we should just use the entire flow for all timestep*fps frames
            flowList.append(np.mean(flow))

        else:
            prevgray = cv2.cvtColor(frameImg, cv2.COLOR_BGR2GRAY)
            flowList.append(0)

        # Contrast & Entropy Capture
        contrastList.append(prevgray.std()) #RMS contrast
        entropyList.append(shannon_entropy(prevgray)) #Shannon Entropy

        # Display
        #cv2.imshow('frame', hsv)
        #if(elapsedSeconds > 0):
        #    cv2.imshow('flow', draw_flow(gray, flow))
        #    cv2.imshow('flow HSV', draw_hsv(flow))

        if cv2.waitKey(1) == ord("q"):
            break

    # Create Data Frames

    motionDataFrame = pd.DataFrame()
    motionDataFrame['Mean'] = meanFlowList
    motionDataFrame['Std'] = stdDevFlowList
    motionDataFrame['Skweness'] = skewnessFlowList
    motionDataFrame['Kurtosis'] = kurtosisFlowList
    motionDataFrame['Max'] = maxFlowList
    motionDataFrame['Min'] = minFlowList
    motionDataFrame['Var'] = varianceFlowList

    contrastDataFrame = pd.DataFrame()
    contrastDataFrame['Mean'] = meanContrastList
    contrastDataFrame['Std'] = stdDevContrastList
    contrastDataFrame['Skweness'] = skewnessContrastList
    contrastDataFrame['Kurtosis'] = kurtosisContrastList
    contrastDataFrame['Max'] = maxContrastList
    contrastDataFrame['Min'] = minContrastList
    contrastDataFrame['Var'] = varianceContrastList

    entropyDataFrame = pd.DataFrame()
    entropyDataFrame['Mean'] = meanEntropyList
    entropyDataFrame['Std'] = stdDevEntropyList
    entropyDataFrame['Skweness'] = skewnessEntropyList
    entropyDataFrame['Kurtosis'] = kurtosisEntropyList
    entropyDataFrame['Max'] = maxEntropyList
    entropyDataFrame['Min'] = minEntropyList
    entropyDataFrame['Var'] = varianceEntropyList

    hueDataFrame = pd.DataFrame.from_dict(hueList, orient='columns')

    satDataFrame = pd.DataFrame.from_dict(satList, orient='columns')

    brightDataFrame = pd.DataFrame.from_dict(brightList, orient='columns')


    # Write to CSV
    videoStatsDirectory = '/content/drive/MyDrive/Tese/CSV_Data/VIDEOS/'+title+'/'

    motionDataFrame.to_csv(videoStatsDirectory+title+"_motion.csv")
    contrastDataFrame.to_csv(videoStatsDirectory+title+"_contrast.csv")
    entropyDataFrame.to_csv(videoStatsDirectory+title+"_entropy.csv")
    hueDataFrame.to_csv(videoStatsDirectory+title+"_hue.csv")
    satDataFrame.to_csv(videoStatsDirectory+title+"_sat.csv")
    brightDataFrame.to_csv(videoStatsDirectory+title+"_bright.csv")

    video.release()
    cv2.destroyAllWindows()

    print("ended at: ", elapsedSeconds,"  Minutes: ", int(elapsedSeconds/60))
    print("----------------------------------------------------------------")

    totalMinutes += int(elapsedSeconds/60)

deepLearningFeatures = np.array(deepLearningList)
#print(deepLearningFeatures)
print(deepLearningFeatures.shape)

np.save("/content/drive/MyDrive/Tese/CSV_Data/FEATURES/deepLearningFeatures", deepLearningFeatures) #adds .npy file extension

print(totalMinutes)

#deepLearningFeatures = np.load("/content/drive/MyDrive/Tese/CSV_Data/FEATURES/deepLearningFeatures.npy")

/content/drive/MyDrive/Tese/RECORDINGS/0.mp4 29 3480.0
     TIMESTEP:  1.0
     TIMESTEP:  2.0
     TIMESTEP:  3.0
     TIMESTEP:  4.0
     TIMESTEP:  5.0
     TIMESTEP:  6.0
     TIMESTEP:  7.0
     TIMESTEP:  8.0
     TIMESTEP:  9.0
     TIMESTEP:  10.0
     TIMESTEP:  11.0
     TIMESTEP:  12.0
     TIMESTEP:  13.0
     TIMESTEP:  14.0
     TIMESTEP:  15.0
     TIMESTEP:  16.0
     TIMESTEP:  17.0
     TIMESTEP:  18.0
     TIMESTEP:  19.0
     TIMESTEP:  20.0
     TIMESTEP:  21.0
     TIMESTEP:  22.0
     TIMESTEP:  23.0
     TIMESTEP:  24.0
     TIMESTEP:  25.0
     TIMESTEP:  26.0
     TIMESTEP:  27.0
     TIMESTEP:  28.0
     TIMESTEP:  29.0
ended at:  1740.0   Minutes:  29
----------------------------------------------------------------
/content/drive/MyDrive/Tese/RECORDINGS/1.mp4 20 2400.0
     TIMESTEP:  1.0
     TIMESTEP:  2.0
     TIMESTEP:  3.0
     TIMESTEP:  4.0
     TIMESTEP:  5.0
     TIMESTEP:  6.0
     TIMESTEP:  7.0
     TIMESTEP:  8.0
     TIMESTEP:  9.0
     TIMESTE

### Features

##### Now that every valid experiment and their labels are in an easily digestible format, we can start building the features. This will be done by merging every one of the pre-processed csv files contained in the CSV_Data folder into one single digestible format

In [ ]:
# MSSQ and SSQ stats
statsMSSQ_DF = pd.read_csv(rootFolder+"/CSV_Data/STATS/PROCESSED_MSSQ.csv")
#statsSSQ_DF = pd.read_csv(rootFolder+"/CSV_Data/STATS/PROCESSED_SSQ.csv")

# Create a DataFrame to hold the features of all experiments 
allFeaturesDF = pd.DataFrame()

labels = pd.read_csv(rootFolder+"/CSV_Data/LABELS/labels.csv")
keys = labels.columns

for key in keys:

  # Create a DataFrame to hold the features of the experiment
  featuresDF = pd.DataFrame()

  # Adding Inputs features (first modifying the 'Minute' column as they start from 1)
  inputsDF = pd.read_csv(rootFolder+"/CSV_Data/INPUTS/INPUTS-TRAD/"+key+".csv")

  duration = inputsDF['Minute'].values[-1]
  featuresDF['Minute'] = pd.Series(range(duration)) # Add Minute column, starting on 0
  featuresDF.insert(0,'Label',fmsTagsDict[key])
  featuresDF.insert(0,'Id',key) # Add Id column to the start

  inputsDF = inputsDF.drop(columns=['Minute'])
  featuresDF = pd.concat([featuresDF,inputsDF],axis=1)

  # Adding Video features (first removing an unnecessary 'Unnamed 0:' column)
  hueDF = pd.read_csv(rootFolder+"/CSV_Data/VIDEOS/"+key+"/"+key+"_hue.csv").iloc[: , 1:]
  satDF = pd.read_csv(rootFolder+"/CSV_Data/VIDEOS/"+key+"/"+key+"_sat.csv").iloc[: , 1:]
  brightDF = pd.read_csv(rootFolder+"/CSV_Data/VIDEOS/"+key+"/"+key+"_bright.csv").iloc[: , 1:]

  """ The less error prone way to do this would be to do 3 seperate loops for each
      color dataframe's columns since they could be different. But they aren't. 
      So I just use the same columns of hueDF for all 3 of them"""
  for columnName, columnData in hueDF.iteritems():
    hueDF.rename(columns = {columnName:"hue_"+columnName}, inplace = True)
    satDF.rename(columns = {columnName:"sat_"+columnName}, inplace = True)
    brightDF.rename(columns = {columnName:"bright_"+columnName}, inplace = True)

  featuresDF = pd.concat([featuresDF,hueDF],axis=1)
  featuresDF = pd.concat([featuresDF,satDF],axis=1)
  featuresDF = pd.concat([featuresDF,brightDF],axis=1)

  motionDF = pd.read_csv(rootFolder+"/CSV_Data/VIDEOS/"+key+"/"+key+"_motion.csv").iloc[: , 1:]
  contrastDF = pd.read_csv(rootFolder+"/CSV_Data/VIDEOS/"+key+"/"+key+"_contrast.csv").iloc[: , 1:]
  entropyDF = pd.read_csv(rootFolder+"/CSV_Data/VIDEOS/"+key+"/"+key+"_entropy.csv").iloc[: , 1:]

  """ Same as before, I know they have the same columns, so I can just do this"""
  for columnName, columnData in motionDF.iteritems():
    motionDF.rename(columns = {columnName:"motion_"+columnName}, inplace = True)
    contrastDF.rename(columns = {columnName:"contrast_"+columnName}, inplace = True)
    entropyDF.rename(columns = {columnName:"entropy_"+columnName}, inplace = True)

  featuresDF = pd.concat([featuresDF,motionDF],axis=1)
  featuresDF = pd.concat([featuresDF,contrastDF],axis=1)
  featuresDF = pd.concat([featuresDF,entropyDF],axis=1)

  # Adding Stats features
  mssqRawScore = statsMSSQ_DF[statsMSSQ_DF["Id"] == int(key)]["MSSQ Raw score"].iloc[0]
  #ssqTotalScore = statsSSQ_DF[statsSSQ_DF["Id"] == int(key)]["TotalScore"].iloc[0]

  featuresDF["MSSQ"] = mssqRawScore
  #featuresDF["SSQ"] = ssqTotalScore

  filename = rootFolder+"/CSV_Data/FEATURES/" + key + ".csv"
  print(filename)
  print(featuresDF.shape)
  featuresDF.to_csv(filename, encoding='utf-8', index=False)

  allFeaturesDF = pd.concat([allFeaturesDF, featuresDF])

datasetDF = allFeaturesDF.drop(columns=['Minute','Id'])
allFeaturesDF = allFeaturesDF.drop(columns=['Label'])

filename = rootFolder+"/CSV_Data/FEATURES/allFeatures.csv"
print(filename)
print(allFeaturesDF.shape)
allFeaturesDF.to_csv(filename, encoding='utf-8', index=False)

filename = rootFolder+"/CSV_Data/FEATURES/dataset.csv"
print(filename)
print(datasetDF.shape)
datasetDF.to_csv(filename, encoding='utf-8', index=False)

/content/drive/MyDrive/Tese/CSV_Data/FEATURES/0.csv
(29, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/1.csv
(20, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/3.csv
(19, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/7.csv
(18, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/8.csv
(29, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/9.csv
(25, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/10.csv
(28, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/11.csv
(27, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/12.csv
(26, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/13.csv
(24, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/14.csv
(26, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/15.csv
(26, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/19.csv
(34, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/20.csv
(30, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/21.csv
(40, 127)
/content/drive/MyDrive/Tese/CSV_Data/FEATURES/allFeatures.csv

In [ ]:
# Get the video features we generated before
videoDeepLearningFeatures = np.load("/content/drive/MyDrive/Tese/CSV_Data/FEATURES/videoDeepLearningFeatures.npy")

# Create an array to hold the input features of all experiments 
allInputs = []

labels = pd.read_csv("/content/drive/MyDrive/Tese/CSV_Data/LABELS/labels.csv")
keys = labels.columns

for key in keys:
  # Getting input features dataframe for each valid experiment
  inputsDF = pd.read_csv("/content/drive/MyDrive/Tese/CSV_Data/INPUTS/INPUTS-DEEP/"+key+".csv")

  inputsDF = inputsDF.drop(columns=['ElapsedSeconds'])

  # iterate over each 120 input instances for this input data
  inputs = []
  for i in range(0,len(inputsDF)-1,120):
    input = inputsDF.iloc[i:i+120].to_numpy()
    inputs.append(input)

  # Stack them all so we have a (<number_of_minutes>,120,7)
  inputs = np.stack(inputs,axis=0)

  allInputs.append(inputs)

# Join all the input features
allInputs = np.concatenate(allInputs,axis=0)

# Join all the deep learning features
deepFeatures = np.concatenate((allInputs,videoDeepLearningFeatures),axis=2)

print(deepFeatures.shape)

np.save("/content/drive/MyDrive/Tese/CSV_Data/FEATURES/deepLearningFeatures", deepFeatures) #adds .npy file extension

(401, 120, 10)


## Machine Learning

#### Actually using the data we obtained

### Features and Labels Preperation



#### Deep Learning Features and Labels

In [2]:
# Get the Features and the Labels

datasetDF = pd.read_csv(rootFolder+"/CSV_Data/FEATURES/TradFeaturesAndLabels.csv")

deep_y = datasetDF['Label'].to_numpy()
deep_X = np.load(rootFolder+"/CSV_Data/FEATURES/deepLearningFeatures.npy")

deep_X = tf.keras.utils.normalize(deep_X, axis=1)

np.random.seed(42)

# Preparing the datasets:
# 90% for training and 10% for testing
deep_X_train, deep_X_test, deep_y_train, deep_y_test = train_test_split(deep_X, deep_y, test_size=0.10, shuffle=True, random_state=39)

#sizes = datasetDF['Label'].value_counts(sort=1)
#plt.pie(sizes,autopct='%1.1f%%') #60% vs 40%

print("Classes")
print(datasetDF.iloc[:,0].value_counts()) # classes

print("Features")
print(deep_y.shape) # 433 labels
print(deep_X.shape) # (433,120,3) features

# Show how many 0's and 1's are in the training and testing data
print("0:", np.count_nonzero(deep_y_train == 0), "1:", np.count_nonzero(deep_y_train == 1))
print("0:", np.count_nonzero(deep_y_test == 0), "1:", np.count_nonzero(deep_y_test == 1))

Classes
0    229
1    172
Name: Label, dtype: int64
Features
(401,)
(401, 120, 10)
0: 208 1: 152
0: 21 1: 20


#### Traditional Learning Features and Labels

In [3]:
# Get the Features and the Labels

datasetDF = pd.read_csv(rootFolder+"/CSV_Data/FEATURES/TradFeaturesAndLabels.csv")

trad_y = datasetDF['Label'].to_numpy()
trad_X = datasetDF.drop(columns=['Label']).to_numpy()

trad_X = tf.keras.utils.normalize(trad_X, axis=1)

print(trad_X.shape)


# Preparing the datasets:
# 90% for training and 10% for testing
trad_X_train, trad_X_test, trad_y_train, trad_y_test = train_test_split(trad_X, trad_y, test_size=0.10, shuffle=True, random_state=42)

print("Classes")
print(datasetDF.iloc[:,0].value_counts()) # classes

print("Features")
print(trad_y.shape) # 433 labels
print(trad_X.shape) # 433*118 features

(401, 124)
Classes
0    229
1    172
Name: Label, dtype: int64
Features
(401,)
(401, 124)


### Machine Learning Models

##### Here we will use a variety of machine learning models and compare results

#### Deep Learning Models

###### Here we will use a variety of machine learning models and compare results

In [ ]:
# Function for CNN models

def define_CNN( learning_rate=0.001, 
                convolutional_layers=1, 
                dropout=0.05, 
                kernels=16, 
                dense_layers=1, 
                dense_layer_kernels=16,
                kernel_size=(3,3), 
                input_shape=(120,10,1), 
                pool_size=(2,2), 
                batch_normalization=False):

  model = tf.keras.Sequential()
  
  for i in range(convolutional_layers):
    if(i == 0):
      model.add(tf.keras.layers.Conv2D(kernels, kernel_size, activation='relu', strides=(1, 1), padding='same', input_shape=input_shape))
    else:
      model.add(tf.keras.layers.Conv2D(kernels, kernel_size, activation='relu', strides=(1, 1), padding='same'))
    
    if(batch_normalization):
      model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=pool_size, padding='same'))

    if(dropout > 0):
      model.add(tf.keras.layers.Dropout(rate=dropout))
  
  model.add(tf.keras.layers.Flatten())

  for i in range(dense_layers):
    model.add(tf.keras.layers.Dense(dense_layer_kernels, activation='relu'))

    if(batch_normalization):
      model.add(tf.keras.layers.BatchNormalization())

  if(batch_normalization):
      model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=["accuracy"])
  
  return model

In [ ]:
# Function for LSTM-RNN models 

def define_LSTM( learning_rate=0.001, 
                 lstm_layers=1,
                 units=10,
                 dropout=0.05, 
                 dense_layers=1, 
                 dense_layer_kernels=16,
                 input_shape=(120,10), 
                 batch_normalization=False):
  
  model = tf.keras.Sequential()
  
  for i in range(lstm_layers):
    if(i == 0):
      if(i == lstm_layers-1):
        model.add(tf.keras.layers.LSTM(units, activation='relu', input_shape=input_shape))
      else:
        model.add(tf.keras.layers.LSTM(units, activation='relu', return_sequences=True, input_shape=input_shape))

    elif(i == lstm_layers-1):
      model.add(tf.keras.layers.LSTM(units, activation='relu'))
    else:
      model.add(tf.keras.layers.LSTM(units, activation='relu', return_sequences=True))
    
    if(batch_normalization):
      model.add(tf.keras.layers.BatchNormalization())

    if(dropout > 0):
      model.add(tf.keras.layers.Dropout(rate=dropout))
  
  model.add(tf.keras.layers.Flatten())

  for i in range(dense_layers):
    model.add(tf.keras.layers.Dense(dense_layer_kernels, activation='relu'))

    if(batch_normalization):
      model.add(tf.keras.layers.BatchNormalization())

  if(batch_normalization):
      model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=["accuracy"])
  
  return model

##### GridSearch the best parameters for CNN and LSTM

###### CNN 

In [ ]:
# Grid Search CNN

scores = []
totalScores = []

sssplit = StratifiedShuffleSplit(n_splits=10,test_size=0.10,random_state=42)
batch_size = [None,4,8,16,32,64]
epochs =  [7,9,10]

# We'll see that if something performs better with the last entry of a given list,
# we'll check if it perfoms better with the one after

learning_rate = [0.001,0.0001]
convolutional_layers=[1,2] # so for this one, if it perfoms better with 2, we'll try with 3 and so on
dropout=[0.0,0.05,0.10]
kernels=[8,16,32] # same for the other direction. if it performs better with 8, we'll try 4
dense_layers=[1,2]
dense_layer_kernels=[8,16,32]

param_grid = dict(learning_rate=learning_rate,
                  convolutional_layers=convolutional_layers,
                  dropout=dropout,
                  kernels=kernels,
                  dense_layers=dense_layers,
                  dense_layer_kernels=dense_layer_kernels)


cnn_X = deep_X.reshape((deep_X.shape[0],deep_X.shape[1],deep_X.shape[2],1))

for epoch in epochs:
  for batch in batch_size:
    model = KerasClassifier(build_fn = define_CNN,
                                epochs=epoch,
                                batch_size=batch,
                                verbose=1)
    clf = GridSearchCV(estimator=model,param_grid=param_grid,cv=sssplit, return_train_score=False)
    clf.fit(cnn_X,deep_y)
    scores.append({
        'model' : 'CNN',
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_,
        'epochs' : epoch,
        'batch_size' : batch
    })

    print(scores)
    totalScores.append(scores)

best_score = None
maxScore = 0

for Scores in totalScores:
  for score in Scores:
    if score['best_score'] >= maxScore:
      best_score = score

best_score

###### LSTM

In [ ]:
# Grid Search LSTM

scores = []
totalScores = []

sssplit = StratifiedShuffleSplit(n_splits=10,test_size=0.10,random_state=42)
batch_size = [None,4,8,16,32,64]
epochs =  [5,10,20]

# We'll see that if something performs better with the last entry of a given list,
# we'll check if it perfoms better with the one after

learning_rate = [0.001,0.0001]
units = [10,25,50,75,100]
lstm_layers=[1,2] # so for this one, if it perfoms better with 2, we'll try with 3 and so on
dropout=[0.0,0.05,0.10]
dense_layers=[1,2]
dense_layer_kernels=[8,16,32]

param_grid = dict(learning_rate=learning_rate,
                  units=units,
                  lstm_layers=lstm_layers,
                  dropout=dropout,
                  dense_layers=dense_layers,
                  dense_layer_kernels=dense_layer_kernels)


for epoch in epochs:
  for batch in batch_size:
    model = KerasClassifier(build_fn = define_LSTM,
                                epochs=epoch,
                                batch_size=batch,
                                verbose=1)
    clf = GridSearchCV(estimator=model,param_grid=param_grid,cv=sssplit,return_train_score=False,verbose=1)
    clf.fit(deep_X,deep_y)
    scores.append({
        'model' : 'CNN',
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_,
        'epochs' : epoch,
        'batch_size' : batch
    })

    print(scores)
    totalScores.append(scores)

best_score = None
maxScore = 0

for Scores in totalScores:
  for score in Scores:
    if score['best_score'] >= maxScore:
      best_score = score

best_score

##### Cross Validation 

###### CNN 

In [ ]:
"""
From the GridSearch, we learned:

36 models achieved a 'best_score': 0.6585365891456604

'best_params': 

{
  'learning_rate': ALWAYS 0.001
  'convolutional_layers': ALWAYS 1,
  'dropout': 0.05 OR 0.1
  'kernels': 8 OR 16
  'dense_layers': ALWAYS 1,
  'dense_layer_kernels': ALWAYS 16,
}, 

'epochs': ALWAYS 10
'batch_size': 4 OR 64
"""

# set early stopping criteria
pat = 5 #this is the number of epochs with no improvment after which the training will stop
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

# define the model checkpoint callback -> this will keep on saving the model as a physical file
#model_checkpoint = tf.keras.callbacks.ModelCheckpoint('fas_mnist_1.h5', verbose=1, save_best_only=True)

# reshape the feature dataset so it satisfies CNN input
cnn_X = deep_X.reshape((deep_X.shape[0],deep_X.shape[1],deep_X.shape[2],1))

epochs = 10
batch_size = 4
numLoops = 10
verbose = 0

total_results = []
total_losses = []

# Run a StratifiedShuffleSplit 10 times on the dataset to approximate randomness

for i in range(numLoops):
  print(i)

  sssplit = StratifiedShuffleSplit(n_splits=10,test_size=0.10)

  results = []
  losses = []
  for train, test in sssplit.split(cnn_X, deep_y):
    # Create the model with the best parameters
    model = define_CNN(learning_rate=0.001, 
                     convolutional_layers=1, 
                     dropout=0.1, 
                     kernels=16, 
                     dense_layers=1, 
                     dense_layer_kernels=16,
                     kernel_size=(3,3), 
                     input_shape=(120,10,1), 
                     pool_size=(2,2), 
                     batch_normalization=False)

    # Fit the model
    model.fit(cnn_X[train], deep_y[train], epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stopping],  verbose=verbose)

    # evaluate the model
    scores = model.evaluate(cnn_X[test], deep_y[test], verbose=verbose)
    #print(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1])

    results.append(scores[1])
    losses.append(scores[0])

  #print("CNN-Mean - ", "loss:", np.mean(losses), "accuracy:", np.mean(results))
  #print("CNN-STD - ", "loss:", np.std(losses), "accuracy:", np.std(results))
  #print("CNN-Best - ", "min-loss:", np.amin(losses), "max-accuracy:", np.amax(results))

  total_results.append(results)
  total_losses.append(losses)

totals = []

for results in total_results:
  for result in results:
    totals.append(result)

print("CNN-ACCURACY")
print("CNN-Mean:",np.mean(totals))
print("CNN-STD:",np.std(totals))
print("CNN-Best:",np.amax(totals))

totals = []

for results in total_losses:
  for result in results:
    totals.append(result)

print("CNN-LOSS")
print("CNN-Mean:",np.mean(totals))
print("CNN-STD:",np.std(totals))
print("CNN-Best:",np.amin(totals))

###### LSTM

In [ ]:
# set early stopping criteria
pat = 5 #this is the number of epochs with no improvment after which the training will stop
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

# define the model checkpoint callback -> this will keep on saving the model as a physical file
#model_checkpoint = tf.keras.callbacks.ModelCheckpoint('fas_mnist_1.h5', verbose=1, save_best_only=True)

epochs = 20
batch_size = 32
numLoops = 10
verbose = 0

total_results = []
total_losses = []

# Run a StratifiedShuffleSplit 10 times on the dataset to approximate randomness

for i in range(numLoops):
  print(i)

  sssplit = StratifiedShuffleSplit(n_splits=10,test_size=0.10)

  results = []
  losses = []
  for train, test in sssplit.split(deep_X, deep_y):
    # Create the model with the best parameters
    model = define_LSTM(  learning_rate=0.001, 
                          lstm_layers=1,
                          units=32,
                          dropout=0.05, 
                          dense_layers=1, 
                          dense_layer_kernels=16,
                          input_shape=(120,10), 
                          batch_normalization=True)

    # Fit the model
    model.fit(deep_X[train], deep_y[train], epochs=epochs, batch_size=batch_size, validation_split=0.1,  verbose=verbose)

    # evaluate the model
    scores = model.evaluate(deep_X[test], deep_y[test], verbose=verbose)
    print(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1])

    results.append(scores[1])
    losses.append(scores[0])

  #print("LSTM-Mean - ", "loss:", np.mean(losses), "accuracy:", np.mean(results))
  #print("LSTM-STD - ", "loss:", np.std(losses), "accuracy:", np.std(results))
  #print("LSTM-Best - ", "min-loss:", np.amin(losses), "max-accuracy:", np.amax(results))

  total_results.append(results)
  total_losses.append(losses)

  # Show how many 0's and 1's are in the training and testing data
  print("Accuracy of guessing just 0's",np.count_nonzero(deep_y[train] == 0)/(np.count_nonzero(deep_y[train] == 0)+np.count_nonzero(deep_y[train] == 1)))
  print("Accuracy of guessing just 0's",np.count_nonzero(deep_y[train] == 0)/(np.count_nonzero(deep_y[train] == 0)+np.count_nonzero(deep_y[train] == 1)))


totals = []

for results in total_results:
  for result in results:
    totals.append(result)

print("LSTM-ACCURACY")
print("LSTM-Mean:",np.mean(totals))
print("LSTM-STD:",np.std(totals))
print("LSTM-Best:",np.amax(totals))

totals = []

for results in total_losses:
  for result in results:
    totals.append(result)

print("LSTM-LOSS")
print("LSTM-Mean:",np.mean(totals))
print("LSTM-STD:",np.std(totals))
print("LSTM-Best:",np.amin(totals))

0
loss 0.6880882978439331 accuracy 0.5609756112098694
loss 0.6878853440284729 accuracy 0.5609756112098694
loss 0.686397910118103 accuracy 0.5609756112098694
loss 0.6891376972198486 accuracy 0.5609756112098694
loss 0.684434711933136 accuracy 0.5609756112098694
loss 0.684076189994812 accuracy 0.5609756112098694
loss 0.6864786148071289 accuracy 0.5609756112098694
loss 0.6817284822463989 accuracy 0.5609756112098694
loss 0.6818827390670776 accuracy 0.5609756112098694
loss 0.6931195855140686 accuracy 0.5609756112098694
Accuracy of guessing just 0's 0.5722222222222222
Accuracy of guessing just 0's 0.5722222222222222
1
loss 0.6881238222122192 accuracy 0.5609756112098694
loss 0.6864941120147705 accuracy 0.5609756112098694
loss 0.6970040202140808 accuracy 0.5609756112098694
loss 0.6827999353408813 accuracy 0.5609756112098694
loss 0.6838153004646301 accuracy 0.5853658318519592
loss 0.6832473278045654 accuracy 0.5609756112098694
loss 0.6969470381736755 accuracy 0.5121951103210449
loss 0.6813243031

##### Feature Reduction 

#### Traditional Machine Learning Models

##### GridSearch the best parameters for SVM.SVC, RandomForest and Logistic Regression

In [ ]:
model_params = {
    'svm' : {
        'model' : svm.SVC(),
        'params' : {
            'C' : [0.5,1,5,10,25,50,100,500,1000],
            'kernel' : ['rbf', 'linear', 'poly', 'sigmoid'],
            'gamma' : [1, 0.1, 0.001, 0.0001, 'auto', 'scale'],
            'degree' : [1, 2, 3, 4, 5, 6, 7]
        }
    },
    'logistic_regression' : {
        'model' : LogisticRegression(solver='liblinear', multi_class='auto'),
        'params' : {
            'C' : [0.5,1,5,10,25,50,100,500,1000],
            'penalty' : ['l1', 'l2']
        }
    },
    'random_forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
          'n_estimators' : [1,5,10,25,50,100,250,500,750,1000],
          'criterion' : ['gini', 'entropy', 'log_loss'],
          'max_features' : ['sqrt', 'log2', None]
        }
    }
}

In [ ]:
scores = []

sssplit = StratifiedShuffleSplit(n_splits=10,test_size=0.10,random_state=42)
#sssplit=RepeatedStratifiedKFold(n_splits=5,n_repeats=10,random_state=42)

for model_name, mp in model_params.items():
  clf = GridSearchCV(mp['model'], mp['params'], cv=sssplit, return_train_score=False,verbose=1)
  clf.fit(trad_X,trad_y)
  scores.append({
      'model' : model_name,
      'best_estimator' : clf.best_estimator_,
      'best_score' : clf.best_score_,
      'best_params' : clf.best_params_
  })

scores

##### Cross Validation

###### Having discovered the best parameters for each model, now we perform cross-validation, using StratifiedShuffleSplit, to train and test our models. The results of this process will tell us which model achieves the best accuracy.

In [ ]:
SVCResults = []
LogisticRegressionResults = []
RandomForestsResults = []

for i in range(10):
  print(i)

  #sssplit=RepeatedStratifiedKFold(n_splits=5,n_repeats=10,random_state=42)
  sssplit = StratifiedShuffleSplit(n_splits=10,test_size=0.10)

  modelSVC = svm.SVC(C = scores[0]['best_params']['C'], gamma=scores[0]['best_params']['gamma'], kernel=scores[0]['best_params']['kernel'], degree=scores[0]['best_params']['degree'])
  results=sklearn.model_selection.cross_val_score(modelSVC,trad_X,trad_y,cv=sssplit)
  #print("SVC-Mean:",np.mean(results))
  #print("SVC-STD:",np.std(results))
  #print("SVC-Best:",np.amax(results))
  SVCResults.append(results)


  modelLogisticRegression = LogisticRegression(solver='liblinear', multi_class='auto', C = scores[1]['best_params']['C'])
  results=sklearn.model_selection.cross_val_score(modelLogisticRegression,trad_X,trad_y,cv=sssplit)
  #print("LogisticRegression-Mean:",np.mean(results))
  #print("LogisticRegression-STD:",np.std(results))
  #print("LogisticRegression-Best:",np.amax(results))
  LogisticRegressionResults.append(results)

  modelRandomForest = RandomForestClassifier(n_estimators=scores[2]['best_params']['n_estimators'])
  results=sklearn.model_selection.cross_val_score(modelRandomForest,trad_X,trad_y,cv=sssplit)
  #print("RandomForest-Mean:",np.mean(results))
  #print("RandomForest-STD:",np.std(results))
  #print("RandomForest-Best:",np.amax(results))
  RandomForestsResults.append(results)

totals = []

for results in SVCResults:
  for result in results:
    totals.append(result)

print("SVC-Mean:",np.mean(totals))
print("SVC-STD:",np.std(totals))
print("SVC-Best:",np.amax(totals))

print()

totals = []

for results in LogisticRegressionResults:
  for result in results:
    totals.append(result)

print("LogisticRegression-Mean:",np.mean(totals))
print("LogisticRegression-STD:",np.std(totals))
print("LogisticRegression-Best:",np.amax(totals))

print()

totals = []

for results in RandomForestsResults:
  for result in results:
    totals.append(result)

print("RandomForests-Mean:",np.mean(totals))
print("RandomForests-STD:",np.std(totals))
print("RandomForests-Best:",np.amax(totals))

print()

# Show how many times a certain model beat the others

wins = {'SVC' : 0, 'LR' : 0, 'RF' : 0, 'TIE' : 0}

for i in range(len(SVCResults)):
  for j in range(len(SVCResults[0])):
    if SVCResults[i][j] >= LogisticRegressionResults[i][j] and SVCResults[i][j] >= RandomForestsResults[i][j]:
      wins['SVC']+=1

    if LogisticRegressionResults[i][j] >= SVCResults[i][j] and LogisticRegressionResults[i][j] >= RandomForestsResults[i][j]:
      wins['LR']+=1

    if RandomForestsResults[i][j] >= LogisticRegressionResults[i][j] and RandomForestsResults[i][j] >= SVCResults[i][j]:
      wins['RF']+=1
    else:
      wins['TIE']+=1
print("single:",wins)

# Show how many times a certain model beat the others but for the means

wins = {'SVC' : 0, 'LR' : 0, 'RF' : 0, 'TIE' : 0}

for i in range(len(SVCResults)):
  if np.mean(SVCResults[i]) >= np.mean(LogisticRegressionResults[i]) and np.mean(SVCResults[i]) >= np.mean(RandomForestsResults[i]):
    wins['SVC']+=1

  if np.mean(LogisticRegressionResults[i]) >= np.mean(SVCResults[i]) and np.mean(LogisticRegressionResults[i]) >= np.mean(RandomForestsResults[i]):
    wins['LR']+=1

  if np.mean(RandomForestsResults[i]) >= np.mean(LogisticRegressionResults[i]) and np.mean(RandomForestsResults[i]) >= np.mean(SVCResults[i]):
    wins['RF']+=1

  else:
    wins['TIE']+=1
print("mean:",wins)

0
1
2
3
4
5
6
7
8
9
SVC-Mean: 0.7109756097560974
SVC-STD: 0.06484090262467533
SVC-Best: 0.8536585365853658

LogisticRegression-Mean: 0.7182926829268291
LogisticRegression-STD: 0.061255455389631036
LogisticRegression-Best: 0.8292682926829268

RandomForests-Mean: 0.7041463414634143
RandomForests-STD: 0.06440640964847566
RandomForests-Best: 0.8292682926829268

single: {'SVC': 36, 'LR': 44, 'RF': 39, 'TIE': 61}
mean: {'SVC': 5, 'LR': 4, 'RF': 2, 'TIE': 8}


##### Features reduction

###### First, we must discover which features have less impact, for each model's best estimator. 

######Then, remove them from the dataset and check the results of cross-validation again. 

######This process will ideally report the best results possible.

In [ ]:
# SVC
# If the best kernel parameter for SVC was non-linear, then we cannot investigate feature importance
# This is because non-linear kernels map the dataset into a space of higher dimension

if(scores[0]['best_params']['kernel'] == 'linear'):
  model = scores[0]['best_estimator']
  prediction_test = model.predict(trad_X_test)
  print("SVC-Accuracy:",sklearn.metrics.accuracy_score(trad_y_test,prediction_test))

  weightsSCVDF = pd.DataFrame(pd.Series(modelSVC.coef_[0], index=datasetDF.drop(columns=['Label']).columns).sort_values(ascending=False)).rename(columns = {0:'feature'})
  weightsSCVDF.to_csv(rootFolder+"/CSV_Data/FEATURES/weightsSCV.csv", encoding='utf-8')

  #sklearn.metrics.ConfusionMatrixDisplay.from_predictions(trad_y_test, prediction_test) #confusion matrix

# Logistic Regression
model = scores[1]['best_estimator']
#model.fit(trad_X_train, trad_y_train)
prediction_test = model.predict(trad_X_test)
print("LogisticRegression-Accuracy:",sklearn.metrics.accuracy_score(trad_y_test,prediction_test))

weightsLogisticRegressionDF = pd.DataFrame(pd.Series(model.coef_[0], index=datasetDF.drop(columns=['Label']).columns).sort_values(ascending=False)).rename(columns = {0:'feature'})
weightsLogisticRegressionDF.to_csv(rootFolder+"/CSV_Data/FEATURES/weightsLogisticRegression.csv", encoding='utf-8')

#sklearn.metrics.ConfusionMatrixDisplay("Logistic Regression").from_predictions(trad_y_test, prediction_test) #confusion matrix

# Random Forest
model = scores[2]['best_estimator']
prediction_test = model.predict(trad_X_test)
print("RandomForest-Accuracy:",sklearn.metrics.accuracy_score(trad_y_test,prediction_test))

weightsRandomForestDF = pd.Series(model.feature_importances_, index=datasetDF.drop(columns=['Label']).columns).sort_values(ascending=False).to_frame().rename(columns = {0:'feature'})
weightsRandomForestDF.to_csv(rootFolder+"/CSV_Data/FEATURES/weightsRandomForest.csv", encoding='utf-8')

#sklearn.metrics.ConfusionMatrixDisplay("Random Forest").from_predictions(trad_y_test, prediction_test) #confusion matrix

LogisticRegression-Accuracy: 0.7560975609756098
RandomForest-Accuracy: 1.0


In [ ]:
# We're going to try different thresholds for removing features

SVCThresholds = [0.25,0.5,1.0,1.5,2.0,2.5,3.0]
LRThresholds = [0.25,0.5,1.0,1.5,2.0,2.5,3.0]
RFThresholds = [0.003,0.004,0.005,0.006,0.007,0.008,0.009]

for k in range(len(SVCThresholds)):
  print("iteration k:",k)

  ######################################################
  # Remove the unnecessary features from the dataset
  ######################################################

  # SVC
  if(scores[0]['best_params']['kernel'] == 'linear'):
    datasetDF = pd.read_csv(rootFolder+"/CSV_Data/FEATURES/dataset.csv")
  
    SVC_y = datasetDF['Label'].to_numpy()
    SVC_X = datasetDF.drop(columns=['Label'])
  
    # Those that have feature importance below 1 and above -1 are probably safe to ignore
    #weightsSCVDF = pd.read_csv(rootFolder+"/CSV_Data/FEATURES/weightsSCV.csv")
    featuresToRemove = weightsSCVDF.loc[(weightsSCVDF['feature'] >= -SVCThresholds[k]) & (weightsSCVDF['feature'] <= SVCThresholds[k])].index.values.tolist()
    SVC_X = SVC_X.drop(columns=featuresToRemove)
  
    SVC_X = tf.keras.utils.normalize(SVC_X, axis=1)  
  
  
  # LogisticRegression
  datasetDF = pd.read_csv(rootFolder+"/CSV_Data/FEATURES/dataset.csv")
  
  LR_y = datasetDF['Label'].to_numpy()
  LR_X = datasetDF.drop(columns=['Label'])
  
  # Those that have feature importance below 1 and above -1 are probably safe to ignore
  #weightsLogisticRegressionDF = pd.read_csv(rootFolder+"/CSV_Data/FEATURES/weightsLogisticRegression.csv")
  featuresToRemove = weightsLogisticRegressionDF.loc[(weightsLogisticRegressionDF['feature'] >= -LRThresholds[k]) & (weightsLogisticRegressionDF['feature'] <= LRThresholds[k])].index.values.tolist()
  LR_X = LR_X.drop(columns=featuresToRemove)
  
  LR_X = tf.keras.utils.normalize(LR_X, axis=1)
    
  # RandomForest
  datasetDF = pd.read_csv(rootFolder+"/CSV_Data/FEATURES/dataset.csv")
  
  RF_y = datasetDF['Label'].to_numpy()
  RF_X = datasetDF.drop(columns=['Label'])
  
  # Those that have feature coeficients that are below 0.006 are probably safe to ignore
  #weightsRandomForestDF = pd.read_csv(rootFolder+"/CSV_Data/FEATURES/weightsRandomForest.csv")
  featuresToRemove = weightsRandomForestDF.loc[(weightsRandomForestDF['feature'] <= RFThresholds[k])].index.values.tolist()
  RF_X = RF_X.drop(columns=featuresToRemove)
  
  RF_X = tf.keras.utils.normalize(RF_X, axis=1)
  



  ######################################################
  # And check the results of cross-validation again
  ######################################################
  SVCResults = []
  LogisticRegressionResults = []
  RandomForestsResults = []

  for i in range(10):
    print(i)

    #sssplit=RepeatedStratifiedKFold(n_splits=5,n_repeats=10,random_state=42)
    sssplit = StratifiedShuffleSplit(n_splits=10,test_size=0.10)

    if(scores[0]['best_params']['kernel'] == 'linear'):
      modelSVC = svm.SVC(C = scores[0]['best_params']['C'], gamma=scores[0]['best_params']['gamma'], kernel=scores[0]['best_params']['kernel'], degree=scores[0]['best_params']['degree'])
      results=sklearn.model_selection.cross_val_score(modelSVC,SVC_X,SVC_y,cv=sssplit)
      #print("SVC-Mean:",np.mean(results))
      #print("SVC-STD:",np.std(results))
      #print("SVC-Best:",np.amax(results))
      SVCResults.append(results)


    modelLogisticRegression = LogisticRegression(solver='liblinear', multi_class='auto', C = scores[1]['best_params']['C'])
    results=sklearn.model_selection.cross_val_score(modelLogisticRegression,LR_X,LR_y,cv=sssplit)
    #print("LogisticRegression-Mean:",np.mean(results))
    #print("LogisticRegression-STD:",np.std(results))
    #print("LogisticRegression-Best:",np.amax(results))
    LogisticRegressionResults.append(results)

    modelRandomForest = RandomForestClassifier(n_estimators=scores[2]['best_params']['n_estimators'])
    results=sklearn.model_selection.cross_val_score(modelRandomForest,RF_X,RF_y,cv=sssplit)
    #print("RandomForest-Mean:",np.mean(results))
    #print("RandomForest-STD:",np.std(results))
    #print("RandomForest-Best:",np.amax(results))
    RandomForestsResults.append(results)



  ######################################################
  # Now check the scores
  ######################################################
  if(scores[0]['best_params']['kernel'] == 'linear'):
    totals = []

    for results in SVCResults:
      for result in results:
        totals.append(result)

    print("SVC-Mean:",np.mean(totals))
    print("SVC-STD:",np.std(totals))
    print("SVC-Best:",np.amax(totals))
    print("SVC-Threshold",SVCThresholds[k])

    print()

  totals = []

  for results in LogisticRegressionResults:
    for result in results:
      totals.append(result)

  print("LogisticRegression-Mean:",np.mean(totals))
  print("LogisticRegression-STD:",np.std(totals))
  print("LogisticRegression-Best:",np.amax(totals))
  print("LogisticRegression-Threshold",LRThresholds[k])
  

  print()

  totals = []

  for results in RandomForestsResults:
    for result in results:
      totals.append(result)

  print("RandomForests-Mean:",np.mean(totals))
  print("RandomForests-STD:",np.std(totals))
  print("RandomForests-Best:",np.amax(totals))
  print("RandomForests-Threshold",RFThresholds[k])

iteration k: 0
0
1
2
3
4
5
6
7
8
9
LogisticRegression-Mean: 0.715121951219512
LogisticRegression-STD: 0.06819505216507822
LogisticRegression-Best: 0.8292682926829268
LogisticRegression-Threshold 0.25

RandomForests-Mean: 0.7214634146341463
RandomForests-STD: 0.06698930455266522
RandomForests-Best: 0.9024390243902439
RandomForests-Threshold 0.003
iteration k: 1
0
1
2
3
4
5
6
7
8
9
LogisticRegression-Mean: 0.7202439024390245
LogisticRegression-STD: 0.06077577008664903
LogisticRegression-Best: 0.8292682926829268
LogisticRegression-Threshold 0.5

RandomForests-Mean: 0.7151219512195119
RandomForests-STD: 0.05913112703487088
RandomForests-Best: 0.8292682926829268
RandomForests-Threshold 0.004
iteration k: 2
0
1
2
3
4
5
6
7
8
9
LogisticRegression-Mean: 0.7185365853658535
LogisticRegression-STD: 0.06832925849827766
LogisticRegression-Best: 0.8536585365853658
LogisticRegression-Threshold 1.0

RandomForests-Mean: 0.7282926829268292
RandomForests-STD: 0.06256656181972685
RandomForests-Best: 0.878

### Final Results

#### Traditional Machine Learning Models

**Best parameters for each model were:**

```
SVC:
* best_params: {'C': 50, 'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}

LogisticRegression:
* best_params: {'C': 100}

RandomForests:
* best_params: {'n_estimators': 1000}
```

**Score for cross-validation were:**
```
SVC:
* SVC-Mean: 0.7109756097560974
* SVC-STD: 0.06484090262467533
* SVC-Best: 0.8536585365853658

LogisticRegression:
* LogisticRegression-Mean: 0.7182926829268291
* LogisticRegression-STD: 0.061255455389631036
* LogisticRegression-Best: 0.8292682926829268

RandomForests:
* RandomForests-Mean: 0.7041463414634143
* RandomForests-STD: 0.06440640964847566
* RandomForests-Best: 0.8292682926829268

Wins Count:
* single: {'SVC': 36, 'LR': 44, 'RF': 39, 'TIE': 61}
* mean: {'SVC': 5, 'LR': 4, 'RF': 2, 'TIE': 8}
```

**After feature reduction, the best results were:**
```
(None for SVC due to non-linear kernel mapping the features to higher dimension)

LogisticRegression: (Nearly a 0.01 increase from before)
* LogisticRegression-Mean: 0.7202439024390244
* LogisticRegression-STD: 0.06420288706835509
* LogisticRegression-Best: 0.8780487804878049
* LogisticRegression-Threshold 1.5


RandomForest (Nearly a 0.03 increase from before)
* RandomForests-Mean: 0.7285365853658538
* RandomForests-STD: 0.06541452501764237
* RandomForests-Best: 0.926829268292683
* RandomForests-Threshold 0.007
```


#### Deep Learning Models

**Best parameters for each model were:**

```
CNN:
best_params: 
* {'learning_rate': ALWAYS 0.001
  'convolutional_layers': ALWAYS 1,
  'dropout': 0.05 (OR 1)
  'kernels': 16 (OR 8)
  'dense_layers': ALWAYS 1,
  'dense_layer_kernels': ALWAYS 16} 

* 'epochs': ALWAYS 10
* 'batch_size': 4 (OR 64)
* 'early stop patience': 5 (for val_loss with val_split=0.1)

LSTM:
best_params: 
* {'learning_rate': ALWAYS 0.001
  'lstm_layers': ALWAYS 1,
  'units' : 32
  'dropout': 0.05 (OR 0.1)
  'dense_layers': ALWAYS 1,
  'dense_layer_kernels': ALWAYS 16} 

* 'epochs': 20
* 'batch_size': 4 (OR 64)
* 'early stop patience': 5 (for val_loss with val_split=0.1)
```

**Score for cross-validation were:**
```
Accuracy of guessing just 0's 0.5722222222222222
(given that the dataset was split with a stratified shuffle split)

CNN:
--ACCURACY
* CNN-Mean: 0.6004878053069115
* CNN-STD: 0.07058690043299648
* CNN-Best: 0.7560975551605225
-- LOSS
* CNN-Mean: 0.7772195053100586
* CNN-STD: 0.1319117559551849
* CNN-Best: 0.5428411960601807


LSTM:
-- ACCURACY
* LSTM-Mean: 0.5575609764456749
* LSTM-STD: 0.030855344448556508
* LSTM-Best: 0.6341463327407837
--LOSS
* LSTM-Mean: 0.690781815648079
* LSTM-STD: 0.010798006145251957
* LSTM-Best: 0.6761513352394104
```

**After feature reduction, the best results were:**
```
There was no feature reduction
```
